In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!which python

/usr/bin/python


In [3]:
from dotenv import load_dotenv
import os
load_dotenv("../.env")

from huggingface_hub import login
from datasets import Dataset, load_dataset, DatasetDict
login(os.getenv("HF_TOKEN"))

import sys
sys.path.append("..")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
from walledeval.judge import LionGuardJudge

In [11]:
binary = LionGuardJudge.from_preset("binary")
binary

In [15]:
binary.embed("Hello World")

array([[ 0.0343454 ,  0.03316101,  0.02191255, ..., -0.00717639,
        -0.00188533,  0.01511723]], dtype=float32)

In [17]:
binary.check("Hello World")

0.07944314947105535

In [19]:
binary.check("bloody fuck you bloody")

0.9448972910301721

In [34]:
binary.check("knn")

0.6833139548316369